In [1]:
import os, sys

project_root = os.path.abspath('/Users/subhojit/workspace/saturn/src')
if project_root not in sys.path:
    sys.path.append(project_root)

from rnn.recnet_batch import *
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
words = open('indian_names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
chars = ['<PAD>', '<SOS>', '<EOS>'] + chars
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(stoi)
print(f"vocab: {stoi}")
print(max(len(w) for w in words))

vocab: {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
19


In [3]:
def encode_name(name):
    encode = [stoi['<SOS>']] + [stoi[ch] for ch in name] + [stoi['<EOS>']]
    return encode

def decode_name(indices):
    return ''.join([itos[i] for i in indices])

encode_name('bobby')

[1, 4, 17, 4, 4, 27, 2]

In [4]:
training_data = []

np.random.shuffle(words)

for name in words:
    # print(name)
    encoded = encode_name(name)

    inputs = []
    targets = []
    for i in range(len(encoded) - 1):
        x_idx = encoded[i]
        y_idx = encoded[i+1]
        x_onehot = np.zeros((vocab_size, 1))
        x_onehot[x_idx] = 1.0
        inputs.append(x_onehot)
        targets.append(y_idx)

    training_data.append((inputs, targets))

print(len(training_data))

6485


In [5]:
def pad_inputs(sequences, pad_vector):
    max_len = max(len(seq) for seq in sequences)
    padded = []

    for seq in sequences:
        padded_seq = seq + [pad_vector] * (max_len - len(seq))
        padded.append(padded_seq)

    return np.array(padded)

def pad_target(sequences, pad_idx):
    max_len = max(len(seq) for seq in sequences)
    padded = []

    for seq in sequences:
        padded_seq = seq + [pad_idx] * (max_len - len(seq))
        padded.append(padded_seq)

    return np.array(padded)

n_epochs = 1000
batch_size = 32  # Set batch size
hidden_size = 128

rnn = VanillaBatchRNN(input_size=vocab_size, hidden_size=hidden_size, output_size=vocab_size)

PAD_IDX = stoi['<PAD>']
pad_vector = np.zeros((vocab_size, 1))
pad_vector[PAD_IDX] = 1.0

for epoch in range(n_epochs):
    total_loss = 0.0

    for i in range(0, len(training_data), batch_size):
        batch = training_data[i:i+batch_size]

        batch_inputs, batch_targets = zip(*batch)

        # Pad to same length
        batch_inputs_padded = pad_inputs(batch_inputs, pad_vector)
        batch_targets_padded = pad_target(batch_targets, PAD_IDX)

        # Now batch_inputs_padded: (batch_size, seq_len)
        # transpose to (seq_len, batch_size)
        batch_inputs_tensor = np.transpose(batch_inputs_padded, (1, 0, 2, 3))
        batch_targets_tensor = np.transpose(batch_targets_padded, (1, 0))

        loss = rnn.train_step(batch_inputs_tensor, batch_targets_tensor)
        total_loss += loss

    avg_loss = total_loss / len(training_data)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")

/Users/subhojit/workspace/saturn/src/rnn/recnet_batch.py:30: RuntimeWarning: divide by zero encountered in matmul
  activation = xt @ self.Wxh.T + ht @ self.Whh.T + self.bh.T  # (batch_size, hidden_size)
/Users/subhojit/workspace/saturn/src/rnn/recnet_batch.py:30: RuntimeWarning: overflow encountered in matmul
  activation = xt @ self.Wxh.T + ht @ self.Whh.T + self.bh.T  # (batch_size, hidden_size)
/Users/subhojit/workspace/saturn/src/rnn/recnet_batch.py:30: RuntimeWarning: invalid value encountered in matmul
  activation = xt @ self.Wxh.T + ht @ self.Whh.T + self.bh.T  # (batch_size, hidden_size)
/Users/subhojit/workspace/saturn/src/rnn/recnet_batch.py:32: RuntimeWarning: divide by zero encountered in matmul
  y = ht @ self.Why.T + self.by.T  # (batch_size, output_size)
/Users/subhojit/workspace/saturn/src/rnn/recnet_batch.py:32: RuntimeWarning: overflow encountered in matmul
  y = ht @ self.Why.T + self.by.T  # (batch_size, output_size)
/Users/subhojit/workspace/saturn/src/rnn/recnet

IndexError: list index out of range